In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/thesissssss
# %pwd

In [4]:
# import libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

In [6]:
%cd ..

/home/sharaf/Desktop/kholy/random_tests


In [7]:
%ls

nfcic2018/  notebooks/


In [8]:
%ls nfcic2018

 cleaned_dataset.csv        NF-CSE-CIC-IDS2018.csv
'NetFlow v2 Features.csv'   Untitled.ipynb


In [9]:
label2int = {'Benign': 0,
 'Bot': 1,
 'Brute Force -Web': 2,
 'Brute Force -XSS': 3,
 'DDOS attack-HOIC': 4,
 'DDOS attack-LOIC-UDP': 5,
 'DDoS attacks-LOIC-HTTP': 6,
 'DoS attacks-GoldenEye': 7,
 'DoS attacks-Hulk': 8,
 'DoS attacks-SlowHTTPTest': 9,
 'DoS attacks-Slowloris': 10,
 'FTP-BruteForce': 11,
 'Infilteration': 12,
 'SQL Injection': 13,
 'SSH-Bruteforce': 14
}

int2label = {0: 'Benign',
 1: 'Bot',
 2: 'Brute Force -Web',
 3: 'Brute Force -XSS',
 4: 'DDOS attack-HOIC',
 5: 'DDOS attack-LOIC-UDP',
 6: 'DDoS attacks-LOIC-HTTP',
 7: 'DoS attacks-GoldenEye',
 8: 'DoS attacks-Hulk',
 9: 'DoS attacks-SlowHTTPTest',
 10: 'DoS attacks-Slowloris',
 11: 'FTP-BruteForce',
 12: 'Infilteration',
 13: 'SQL Injection',
 14: 'SSH-Bruteforce'
}


In [10]:
# function to reduce memory usage
# link https://stackoverflow.com/questions/57531266/how-do-i-read-a-large-csv-file-in-google-colab

def process(df):
    # loop over dataframe cols
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            # get min
            col_min = df[col].min()
            col_max = df[col].max()

            # check if col type is int
            if str(col_type)[:3] == "int":
                if col_min > np.iinfo(np.int8).min and col_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif col_min > np.iinfo(np.uint8).min and col_max < np.iinfo(np.uint8).max:
                    df[col] = df[col].astype(np.uint8)
                elif col_min > np.iinfo(np.int16).min and col_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif col_min > np.iinfo(np.uint16).min and col_max < np.iinfo(np.uint16).max:
                    df[col] = df[col].astype(np.uint16)
                elif col_min > np.iinfo(np.int32).min and col_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif col_min > np.iinfo(np.uint32).min and col_max < np.iinfo(np.uint32).max:
                    df[col] = df[col].astype(np.uint32)
                elif col_min > np.iinfo(np.int64).min and col_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
                elif col_min > np.iinfo(np.uint64).min and col_max < np.iinfo(np.uint64).max:
                    df[col] = df[col].astype(np.uint64)
            elif str(col_type)[:5] =="float":
                if col_min > np.finfo(np.float16).min and col_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif col_min > np.finfo(np.float32).min and col_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    return df

cols_to_drop = ["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "L4_SRC_PORT", "Label"]
def reduce_mem_usage(file_name, drop_cols = True, map_labels=True):
    gc.collect()
    # create an iterator over df
    df_iter = pd.read_csv(file_name, chunksize=10000)
    df_list = []

    for df in df_iter:
        if drop_cols:
            df = df.drop(columns=cols_to_drop)
        if map_labels:
            df["Attack"] = df["Attack"].map(lambda x: label2int[x]).astype(np.int8)
        df_list.append(process(df))

    df_list = pd.concat(df_list)
    gc.collect()
    mem = df_list.memory_usage().sum()/1024**2
    print(f"Memory usage after processing: {mem}")
    return df_list

# nfcic2018 = reduce_mem_usage("nfcic2018/NF-CSE-CIC-IDS2018-v2.csv")

In [ ]:
# # get label encoder
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# labels = le.fit_transform(nfcic2018["Attack"])
# int2label = {label: class_name for label, class_name in enumerate(le.classes_)}
# label2int = {class_name: label for label, class_name in int2label.items()}


In [ ]:
# save to parquet file
# nfcic2018.to_parquet("nfcic2018/smaller_size.parquet")

In [7]:
# nfcic2018

,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack
0,22,6,92.00,3164,23,3765,21,27,27,27,...,1,26883,26847,0,0,0,0,0,0,14
1,3389,6,0.00,1919,14,2031,11,223,219,30,...,1,8192,64000,0,0,0,0,0,0,0
2,53,17,0.00,116,2,148,2,0,0,0,...,0,0,0,0,0,2511,1,5,0,0
3,53,17,0.00,70,1,130,1,0,0,0,...,0,0,0,0,0,3371,1,60,0,0
4,80,6,7.00,232,5,1136,4,223,222,27,...,0,8192,26883,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18893703,53,17,0.00,77,1,138,1,0,0,0,...,0,0,0,0,0,14428,1,60,0,0
18893704,443,6,91.25,1729,20,6719,16,27,27,27,...,3,8192,29200,0,0,0,0,0,0,0
18893705,80,6,7.00,561,5,1147,5,219,219,27,...,0,65535,26883,0,0,0,0,0,0,4
18893706,445,6,41.00,120,3,124,3,23,19,22,...,0,65392,65392,0,0,0,0,0,0,0


#Load nfcic dataset

In [ ]:
# drop duplicates
# nfcic2018.drop_duplicates(inplace=True)

In [11]:
# nfcic2018.reset_index(inplace=True, drop=True)

In [12]:
# nfcic2018

,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack
0,22,6,92.000,3164,23,3765,21,27,27,27,...,1,26883,26847,0,0,0,0,0,0,14
1,3389,6,0.000,1919,14,2031,11,223,219,30,...,1,8192,64000,0,0,0,0,0,0,0
2,53,17,0.000,116,2,148,2,0,0,0,...,0,0,0,0,0,2511,1,5,0,0
3,53,17,0.000,70,1,130,1,0,0,0,...,0,0,0,0,0,3371,1,60,0,0
4,80,6,7.000,232,5,1136,4,223,222,27,...,0,8192,26883,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9037753,53,17,0.000,118,2,246,2,0,0,0,...,0,0,0,0,0,60284,1,60,0,0
9037754,53,17,5.125,63,1,115,1,0,0,0,...,0,0,0,0,0,29031,1,60,0,0
9037755,53,17,0.000,77,1,138,1,0,0,0,...,0,0,0,0,0,14428,1,60,0,0
9037756,443,6,91.250,1729,20,6719,16,27,27,27,...,3,8192,29200,0,0,0,0,0,0,0


In [13]:
# nfcic2018.to_csv("nfcic2018/cleaned_dataset.csv", index=False)

In [11]:
# read csv
nfcic2018 = reduce_mem_usage("nfcic2018/cleaned_dataset.csv", drop_cols=False, map_labels=False)

# nfcic2018.head()

Memory usage after processing: 1215.290075302124


In [8]:
# from collections import Counter

# num_per_class = Counter(nfcic2018["Attack"])
# num_per_class = {int2label[k]: v for k, v in num_per_class.items()}
# print(num_per_class)

In [14]:
# cols_list = nfcic2018.columns.to_list()

In [21]:
# nfcic2018[cols_list[27:32]]

,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES
0,33,7,1,2,1
1,17,6,0,1,1
2,4,0,0,0,0
3,1,1,0,0,0
4,8,0,0,1,0
...,...,...,...,...,...
9037753,4,0,0,0,0
9037754,2,0,0,0,0
9037755,1,1,0,0,0
9037756,26,3,3,1,3


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['NUM_PKTS_UP_TO_128_BYTES'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_1, *['NUM_PKTS_128_TO_256_BYTES'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_2, *['NUM_PKTS_256_TO_512_BYTES'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_3, *['NUM_PKTS_512_TO_1024_BYTES'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_4, *['NUM_PKTS_UP_TO_128_BYTES'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_5, *['NUM_PKTS_128_TO_256_BYTES'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_6, *['NUM_PKTS_256_TO_512_BYTES'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_7, *['NUM_PKTS_512_TO_1024_BYTES'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(_df_8, *[[['NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES'], ['NUM_PKTS_128_TO_256_BYTES', 'NUM_PKTS_256_TO_512_BYTES'], ['NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES'], ['NUM_PKTS_512_TO_1024_BYTES', 'NUM_PKTS_1024_TO_1514_BYTES']]], **{})
chart

In [12]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(nfcic2018, test_size=0.2, random_state=42, stratify=nfcic2018["Attack"])

In [13]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [35]:
train_x = train.iloc[:, :-1].astype(np.float64).to_numpy()
train_y = train.iloc[:, -1].astype(np.int64).to_numpy()
test_x = train.iloc[:, :-1].astype(np.float64).to_numpy()
test_y = train.iloc[:, -1].astype(np.int64).to_numpy()

In [43]:
train_x

array([[53.        , 17.        ,  0.        , ...,  1.        ,
         7.        ,  0.        ],
       [53.        , 17.        ,  0.        , ...,  1.        ,
        11.        ,  0.        ],
       [53.        , 17.        ,  0.        , ...,  1.        ,
        43.        ,  0.        ],
       ...,
       [80.        ,  6.        ,  7.        , ...,  0.        ,
         0.        ,  0.        ],
       [53.        , 17.        ,  0.        , ...,  1.        ,
        60.        ,  0.        ],
       [53.        , 17.        ,  5.23828125, ...,  1.        ,
        60.        ,  0.        ]])

In [46]:
# check if there are any nans or infs in numpy array
# check if any infs in numpy array
np.isinf(train_x).any(), np.isinf(train_y).any(), np.isinf(test_x).any(), np.isinf(test_y).any()

(False, False, False, False)

In [54]:
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

rf = RandomForestClassifier(n_estimators=150, n_jobs=-1)
rf.fit(train_x, train_y)
# sel = SelectFromModel(RandomForestClassifier(n_estimators=150))
# sel.fit(train_x, train_y)

/home/sharaf/miniconda3/envs/nlp/lib/python3.9/site-packages/sklearn/utils/_array_api.py:380: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)


ValueError: Input X contains infinity or a value too large for dtype('float32').